# Extração de Dados com Selenium

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def slow_scroll():
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.END)
    time.sleep(1)

url = 'https://www.rottentomatoes.com/m/madame_web/reviews?type=user'
option = Options()
option.headless = False
driver = webdriver.Firefox(options=option)
driver.get(url)

In [2]:
# Lista para armazenar os produtos de todas as páginas
reviews = []

In [3]:
# Loop para percorrer até o final
while True:
    slow_scroll()
    
    div_main = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/div/section/div/div[1]')
    
    html_content = div_main.get_attribute('outerHTML')
    soup = BeautifulSoup(html_content, 'html.parser')
    comentarios = soup.find_all('div', class_='audience-review-row')
    reviews.extend(comentarios)

    # Verificar se há um botão "Load More" e clicar nele
    try:
        load_more = driver.find_element(By.CSS_SELECTOR, '.load-more-container > rt-button:nth-child(1)')
        load_more.click()
        time.sleep(5)  # Esperar um pouco para a próxima página carregar completamente
    except:
        break  # Se não houver botão "Load More", sair do loop

In [4]:
dados_reviews = []

In [5]:
# Coletar texto e nome dos comentários
for review in reviews:
    texto_container = review.find('div', class_='review-text-container')
    nome_container = review.find('div', class_='review-data')

    if texto_container and nome_container:
        texto = texto_container.get_text(strip=True)
        nome = nome_container.get_text(strip=True)
        
        # Adicionar os dados ao array
        dados_reviews.append({'Comentario': texto, 'Nome': nome})

df = pd.DataFrame(dados_reviews)

In [6]:
# df.to_csv("npl_madameteia_raw.csv")